# ConversionCentral Managed Profiling
Run this notebook from a Databricks Repo so backend deployments control profiling logic.

In [ ]:
# Collect parameters passed by the FastAPI backend
# Each widget is declared up front so Databricks jobs can safely supply overrides.
dbutils.widgets.text("table_group_id", "")
dbutils.widgets.text("profile_run_id", "")
dbutils.widgets.text("data_quality_schema", "")
dbutils.widgets.text("payload_path", "")
dbutils.widgets.text("payload_base_path", "")
dbutils.widgets.text("callback_url", "")
dbutils.widgets.text("callback_base_url", "")
dbutils.widgets.text("callback_token", "")
dbutils.widgets.text("payload_storage", "")
dbutils.widgets.text("callback_behavior", "")
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema_name", "")
dbutils.widgets.text("connection_id", "")
dbutils.widgets.text("connection_name", "")
dbutils.widgets.text("system_id", "")
dbutils.widgets.text("project_key", "")
dbutils.widgets.text("http_path", "")

from datetime import datetime
import json
import requests
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
table_group_id = dbutils.widgets.get("table_group_id")
profile_run_id = dbutils.widgets.get("profile_run_id")
dq_schema = (dbutils.widgets.get("data_quality_schema") or "").strip()
raw_payload_path = (dbutils.widgets.get("payload_path") or "").strip()
payload_path = raw_payload_path or None
payload_base_path = (dbutils.widgets.get("payload_base_path") or "").strip() or None
callback_url = (dbutils.widgets.get("callback_url") or "").strip() or None
callback_base_url = (dbutils.widgets.get("callback_base_url") or "").strip() or None
callback_token = (dbutils.widgets.get("callback_token") or "").strip() or None
connection_catalog = (dbutils.widgets.get("catalog") or "").strip()
connection_schema = (dbutils.widgets.get("schema_name") or "").strip()

if not table_group_id or not profile_run_id:
    raise ValueError("Required widgets missing: table_group_id/profile_run_id")
if not dq_schema:
    raise ValueError("Data quality schema widget is required for profiling runs.")

In [ ]:
# Profile the tables registered for this table group and build the result payload.
from datetime import datetime
import re
from contextlib import suppress
from typing import Iterable

import datetime as dt
import hashlib
import json
import math

from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, BinaryType, MapType, StructType
from pyspark.sql import types as T
from pyspark.sql.utils import AnalysisException

MAX_COLUMNS_TO_PROFILE = 25
NULL_RATIO_ALERT_THRESHOLD = 0.5
HIGH_NULL_RATIO_THRESHOLD = 0.9
VALUE_DISTRIBUTION_LIMIT = 25
VALUE_DISTRIBUTION_DISTINCT_THRESHOLD = 1000
VALUE_DISTRIBUTION_MAX_ROWS = 5_000_000
MAX_VALUE_DISPLAY_LENGTH = 256

PROFILE_COLUMN_FIELDS = [
    "profile_run_id",
    "schema_name",
    "table_name",
    "column_name",
    "qualified_name",
    "data_type",
    "general_type",
    "ordinal_position",
    "row_count",
    "null_count",
    "non_null_count",
    "distinct_count",
    "min_value",
    "max_value",
    "avg_value",
    "stddev_value",
    "median_value",
    "p95_value",
    "true_count",
    "false_count",
    "min_length",
    "max_length",
    "avg_length",
    "non_ascii_ratio",
    "min_date",
    "max_date",
    "date_span_days",
    "metrics_json",
    "generated_at",
]

PROFILE_COLUMNS_SCHEMA = T.StructType(
    [
        T.StructField("profile_run_id", T.StringType(), False),
        T.StructField("schema_name", T.StringType(), True),
        T.StructField("table_name", T.StringType(), False),
        T.StructField("column_name", T.StringType(), False),
        T.StructField("qualified_name", T.StringType(), True),
        T.StructField("data_type", T.StringType(), True),
        T.StructField("general_type", T.StringType(), True),
        T.StructField("ordinal_position", T.IntegerType(), True),
        T.StructField("row_count", T.LongType(), True),
        T.StructField("null_count", T.LongType(), True),
        T.StructField("non_null_count", T.LongType(), True),
        T.StructField("distinct_count", T.LongType(), True),
        T.StructField("min_value", T.StringType(), True),
        T.StructField("max_value", T.StringType(), True),
        T.StructField("avg_value", T.DoubleType(), True),
        T.StructField("stddev_value", T.DoubleType(), True),
        T.StructField("median_value", T.DoubleType(), True),
        T.StructField("p95_value", T.DoubleType(), True),
        T.StructField("true_count", T.LongType(), True),
        T.StructField("false_count", T.LongType(), True),
        T.StructField("min_length", T.IntegerType(), True),
        T.StructField("max_length", T.IntegerType(), True),
        T.StructField("avg_length", T.DoubleType(), True),
        T.StructField("non_ascii_ratio", T.DoubleType(), True),
        T.StructField("min_date", T.DateType(), True),
        T.StructField("max_date", T.DateType(), True),
        T.StructField("date_span_days", T.IntegerType(), True),
        T.StructField("metrics_json", T.StringType(), True),
        T.StructField("generated_at", T.TimestampType(), True),
    ]
)

PROFILE_COLUMN_VALUES_FIELDS = [
    "profile_run_id",
    "schema_name",
    "table_name",
    "column_name",
    "value",
    "value_hash",
    "frequency",
    "relative_freq",
    "rank",
    "bucket_label",
    "bucket_lower_bound",
    "bucket_upper_bound",
    "generated_at",
]

PROFILE_COLUMN_VALUES_SCHEMA = T.StructType(
    [
        T.StructField("profile_run_id", T.StringType(), False),
        T.StructField("schema_name", T.StringType(), True),
        T.StructField("table_name", T.StringType(), False),
        T.StructField("column_name", T.StringType(), False),
        T.StructField("value", T.StringType(), True),
        T.StructField("value_hash", T.StringType(), True),
        T.StructField("frequency", T.LongType(), True),
        T.StructField("relative_freq", T.DoubleType(), True),
        T.StructField("rank", T.IntegerType(), True),
        T.StructField("bucket_label", T.StringType(), True),
        T.StructField("bucket_lower_bound", T.DoubleType(), True),
        T.StructField("bucket_upper_bound", T.DoubleType(), True),
        T.StructField("generated_at", T.TimestampType(), True),
    ]
)

PROFILE_ANOMALIES_FIELDS = [
    "profile_run_id",
    "table_name",
    "column_name",
    "anomaly_type",
    "severity",
    "description",
    "detected_at",
]

PROFILE_ANOMALIES_SCHEMA = T.StructType(
    [
        T.StructField("profile_run_id", T.StringType(), False),
        T.StructField("table_name", T.StringType(), True),
        T.StructField("column_name", T.StringType(), True),
        T.StructField("anomaly_type", T.StringType(), True),
        T.StructField("severity", T.StringType(), True),
        T.StructField("description", T.StringType(), True),
        T.StructField("detected_at", T.TimestampType(), True),
    ]
)

def _split_identifier(value: str | None) -> list[str]:
    cleaned = (value or "").replace("`", "").strip()
    if not cleaned:
        return []
    return [segment.strip() for segment in cleaned.split(".") if segment.strip()]


def _catalog_component(value: str | None) -> str | None:
    parts = _split_identifier(value)
    if len(parts) >= 2:
        return parts[0]
    return None

In [ ]:
# Persist payload and call back into the API

from datetime import datetime
import re
import socket
from contextlib import suppress
from functools import lru_cache
from urllib.parse import urlparse, urlunparse

from pyspark.sql import functions as F
from pyspark.sql.utils import AnalysisException

DEFAULT_PRIVATE_PAYLOAD_ROOT = "dbfs:/tmp/conversioncentral/profiles"
DEFAULT_DRIVER_PAYLOAD_ROOT = "file:/databricks/driver/conversioncentral/profiles"
DEFAULT_CALLBACK_BEHAVIOR = "metadata_only"

DEFAULT_PAYLOAD_STORAGE_MODE = "inline"


DBFS_DISABLED_MESSAGES = ("public dbfs root is disabled", "access is denied")
DRIVER_DISABLED_MESSAGES = ("local filesystem access is forbidden", "workspacelocalfilesystem")
URI_SCHEME_PATTERN = re.compile(r"^[a-z][a-z0-9+.\-]*:/", re.IGNORECASE)
_DBFS_REDIRECT_NOTICE_EMITTED = False
_STORAGE_DISABLED_NOTICE_EMITTED = False


def _looks_like_dns_failure(error: BaseException) -> bool:
    """Detect DNS resolution failures from nested request exceptions."""
    current = error
    while current:
        if isinstance(current, socket.gaierror):
            return True
        name = current.__class__.__name__.lower()
        if "nameresolution" in name:
            return True
        message = str(current).lower()
        if "temporary failure in name resolution" in message:
            return True
        current = getattr(current, "__cause__", None) or getattr(current, "__context__", None)
    return False


def _rewrite_heroku_app_host(url: str | None) -> str | None:
    """Fallback to canonical Heroku hostname when review-app hosts fail DNS."""
    if not url:
        return None
    parsed = urlparse(url)
    host = parsed.hostname
    if not host:
        return None
    match = re.match(r"^(?P<base>[a-z0-9-]+?)-[0-9a-f]{12}\.herokuapp\.com$", host)
    if not match:
        return None
    canonical_host = f"{match.group('base')}.herokuapp.com"
    netloc = canonical_host
    if parsed.port:
        netloc = f"{canonical_host}:{parsed.port}"
    if parsed.username:
        auth = parsed.username
        if parsed.password:
            auth = f"{auth}:{parsed.password}"
        netloc = f"{auth}@{netloc}"
    scheme = parsed.scheme or "https"
    if scheme.lower() == "http":
        scheme = "https"
    return urlunparse(parsed._replace(netloc=netloc, scheme=scheme))


def _is_dbfs_path(path: str | None) -> bool:
    return bool(path and path.lower().startswith("dbfs:/"))


def _has_uri_scheme(value: str | None) -> bool:
    return bool(value and URI_SCHEME_PATTERN.match(value.strip()))


@lru_cache(maxsize=1)
def _dbfs_root_is_disabled() -> bool:
    probe_path = f"{DEFAULT_PRIVATE_PAYLOAD_ROOT}/_dbfs_access_probe"
    try:
        dbutils.fs.mkdirs(probe_path)
        dbutils.fs.rm(probe_path, True)
        return False
    except Exception as exc:  # noqa: BLE001 - Databricks surfaces JVM errors generically
        message = str(exc).lower()
        return any(fragment in message for fragment in DBFS_DISABLED_MESSAGES)


@lru_cache(maxsize=1)
def _driver_fs_is_disabled() -> bool:
    probe_path = f"{DEFAULT_DRIVER_PAYLOAD_ROOT}/_driver_access_probe"
    try:
        dbutils.fs.mkdirs(probe_path)
        dbutils.fs.rm(probe_path, True)
        return False
    except Exception as exc:  # noqa: BLE001 - Databricks surfaces JVM errors generically
        message = str(exc).lower()
        return any(fragment in message for fragment in DRIVER_DISABLED_MESSAGES)


def _warn_storage_disabled(message: str) -> None:
    global _STORAGE_DISABLED_NOTICE_EMITTED
    if not _STORAGE_DISABLED_NOTICE_EMITTED:
        print(message)
        _STORAGE_DISABLED_NOTICE_EMITTED = True


def _redirect_dbfs_path(path: str) -> str | None:
    global _DBFS_REDIRECT_NOTICE_EMITTED
    if not _is_dbfs_path(path):
        return path
    if not _dbfs_root_is_disabled():
        return path
    if _driver_fs_is_disabled():
        _warn_storage_disabled(
            "DBFS root access and driver filesystem writes are both disabled; payload artifacts will be skipped unless "
            "a cloud storage payload_base_path is provided."
        )
        return None
    if not _DBFS_REDIRECT_NOTICE_EMITTED:
        print(
            "DBFS root access is disabled on this workspace; persisting profiling artifacts to the driver filesystem "
            "instead."
        )
        _DBFS_REDIRECT_NOTICE_EMITTED = True
    suffix = path[len("dbfs:/") :].lstrip("/")
    redirected = f"{DEFAULT_DRIVER_PAYLOAD_ROOT}/{suffix}" if suffix else DEFAULT_DRIVER_PAYLOAD_ROOT
    return redirected.rstrip("/")


def _mkdirs_if_supported(target_path: str) -> None:
    lowered = target_path.lower()
    if lowered.startswith("dbfs:/") and _dbfs_root_is_disabled():
        return
    if lowered.startswith("file:/") and _driver_fs_is_disabled():
        return
    if lowered.startswith("dbfs:/") or lowered.startswith("file:/"):
        parent_dir = target_path.rsplit("/", 1)[0]
        dbutils.fs.mkdirs(parent_dir)


def _ensure_https_base_url(value: str) -> str:
    normalized = (value or "").strip()
    if not normalized:
        return normalized
    parsed = urlparse(normalized)
    if not parsed.scheme:
        normalized = f"https://{normalized.lstrip('/')}"
        parsed = urlparse(normalized)
    if parsed.scheme.lower() == "http":
        parsed = parsed._replace(scheme="https")
    normalized = urlunparse(parsed).rstrip("/")
    return normalized


def _lookup_metadata_setting(setting_key: str) -> str | None:
    normalized_key = (setting_key or "").strip().lower()
    if not normalized_key:
        return None
    try:
        settings_table = _metadata_table("dq_settings")
    except NameError:
        return None
    try:
        row = (
            spark.table(settings_table)
            .where(F.lower(F.col("key")) == normalized_key)
            .select("value")
            .limit(1)
            .collect()
        )
    except AnalysisException:
        return None
    if not row:
        return None
    value = row[0].get("value")
    return value.strip() if isinstance(value, str) and value.strip() else None




def _normalize_payload_storage_mode(value: str | None) -> str | None:
    normalized = (value or "").strip().lower()
    if not normalized:
        return None
    if normalized in {"inline", "database", "db"}:
        return "inline"
    if normalized in {"artifact", "artifacts", "file", "files", "path", "paths", "dbfs", "cloud"}:
        return "artifact"
    return None


def _resolve_payload_storage_mode() -> str:
    widget_choice = _normalize_payload_storage_mode(dbutils.widgets.get("payload_storage"))
    if widget_choice:
        return widget_choice
    setting_choice = _normalize_payload_storage_mode(_lookup_metadata_setting("profile_payload_storage_mode"))
    if setting_choice:
        return setting_choice
    return DEFAULT_PAYLOAD_STORAGE_MODE


def _payload_storage_is_artifact(mode: str) -> bool:
    return (mode or "").strip().lower() == "artifact"


def _encode_payload_json(payload: dict[str, object]) -> str | None:
    try:
        return json.dumps(payload, separators=(",", ":"))
    except TypeError as exc:
        print(f"Unable to serialize profiling payload: {exc}")
        return None


def _resolve_callback_behavior() -> str:
    widget_value = (dbutils.widgets.get("callback_behavior") or "").strip().lower()
    if widget_value:
        return widget_value
    setting_value = (_lookup_metadata_setting("profile_callback_behavior") or "").strip().lower()
    if setting_value:
        return setting_value
    return DEFAULT_CALLBACK_BEHAVIOR


def _callbacks_enabled(behavior: str) -> bool:
    if behavior in {"api", "callback", "legacy"}:
        return True
    if behavior in {"metadata_only", "metadata", "skip", "disabled", "off"}:
        return False
    print(f"Unknown callback behavior '{behavior}'; defaulting to metadata_only.")
    return False





def _sql_string_literal(value: str | None) -> str:
    if value is None:
        return "NULL"
    escaped = str(value).replace("'", "''")
    return f"'{escaped}'"


def _sql_numeric_literal(value: int | float | None) -> str:
    if value is None:
        return "NULL"
    try:
        return str(int(value))
    except (TypeError, ValueError):
        return "NULL"


def _normalize_temp_view_name(suffix: str | None) -> str:
    cleaned = re.sub(r"[^a-zA-Z0-9_]", "_", (suffix or "profile_run"))
    return f"_profile_anomalies_{cleaned}"


def _parse_anomaly_timestamp(value: str | None) -> datetime | None:
    if not value:
        return None
    candidate = value.strip()
    if not candidate:
        return None
    if candidate.endswith("Z"):
        candidate = f"{candidate[:-1]}+00:00"
    with suppress(ValueError):
        return datetime.fromisoformat(candidate)
    return None


def _persist_results_to_metadata(results_payload: dict[str, object], payload_location: str | None) -> None:
    if not profile_run_id:
        raise ValueError("profile_run_id widget is required before persisting profiling metadata.")
    profiles_table = _metadata_table("dq_profiles")
    anomalies_table = _metadata_table("dq_profile_anomalies")
    assignments = [
        f"status = {_sql_string_literal(results_payload.get('status') or 'unknown')}",
        "completed_at = current_timestamp()",
        f"row_count = {_sql_numeric_literal(results_payload.get('row_count'))}",
        f"anomaly_count = {_sql_numeric_literal(results_payload.get('anomaly_count'))}",
        f"payload_path = {_sql_string_literal(payload_location)}",
    ]
    update_sql = (
        f"UPDATE {profiles_table} "
        f"SET {', '.join(assignments)} "
        f"WHERE profile_run_id = {_sql_string_literal(profile_run_id)}"
    )
    spark.sql(update_sql)
    print(f"Updated dq_profiles entry for run {profile_run_id}.")

    anomalies = list(results_payload.get("anomalies") or [])
    delete_sql = f"DELETE FROM {anomalies_table} WHERE profile_run_id = {_sql_string_literal(profile_run_id)}"
    spark.sql(delete_sql)

    if not anomalies:
        print(f"No anomalies to persist for run {profile_run_id}.")
        return

    anomaly_rows = []
    for anomaly in anomalies:
        anomaly_rows.append(
            {
                "profile_run_id": profile_run_id,
                "table_name": anomaly.get("table_name"),
                "column_name": anomaly.get("column_name"),
                "anomaly_type": anomaly.get("anomaly_type"),
                "severity": anomaly.get("severity"),
                "description": anomaly.get("description"),
                "detected_at": _parse_anomaly_timestamp(anomaly.get("detected_at")) or datetime.utcnow(),
            }
        )

    anomalies_df = spark.createDataFrame(anomaly_rows, PROFILE_ANOMALIES_SCHEMA).select(*PROFILE_ANOMALIES_FIELDS)
    view_name = _normalize_temp_view_name(profile_run_id)
    try:
        anomalies_df.createOrReplaceTempView(view_name)
        spark.sql(
            f"INSERT INTO {anomalies_table} "
            "(profile_run_id, table_name, column_name, anomaly_type, severity, description, detected_at) "
            f"SELECT profile_run_id, table_name, column_name, anomaly_type, severity, description, detected_at FROM {view_name}"
        )
    finally:
        with suppress(Exception):
            spark.catalog.dropTempView(view_name)

    print(f"Persisted {len(anomalies)} anomalies for run {profile_run_id}.")